In [1]:
import sympy as sp

from symqups import s
from symqups.objects.base import PhaseSpaceObject, qpTypePSO, alphaTypePSO
from symqups.objects.scalars import q, p, alpha, alphaD, W, mu, _Primed, _DerivativeSymbol, hbar, _DePrimed, StateFunction
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.utils.algebra import define, qp2a, get_random_poly
from symqups.operations.operator_ordering import sOrdering, _make_normal_ordered
# from symqups.operations.quantization import s_quantize, naive_quantize
from symqups.objects import scalars
from symqups.objects.cache import _sub_cache
from symqups.operations.star_product import Bopp, Star, _replace_diff
from symqups.utils.multiprocessing import _mp_helper
from symqups.utils._internal._operator_handling import _separate_operator
from symqups.utils._internal._operator_handling import (
    _collect_alpha_type_oper_from_monomial, _separate_operator, _separate_by_oper_polynomiality,
    _collect_non_polynomial_by_sub
)


a = alpha()
a_1 = alpha(1)
aop = annihilateOp()
aop_1 = annihilateOp(1)
ad = alphaD()
ad_1 = alphaD(1)
adop = createOp()
adop_1 = createOp(1)
x = sp.Symbol("x")
qq = q()
pp = p()
W = scalars.W
s.val = 0
s

s

In [50]:
expr = sp.exp(3*aop+adop_1)*adop_1**2 * aop_1 * adop**3
expr

exp(3*\hat{a}_{} + \hat{a}^{\dagger}_{1})*\hat{a}^{\dagger}_{1}**2*\hat{a}_{1}*\hat{a}^{\dagger}_{}**3

In [53]:
def _collect_non_polynomial_by_sub(expr : sp.Expr):
    """
    'expr' is a Mul whose 'args' all 'has' 'Operator'.
    """
    if not(isinstance(expr, sp.Mul)):
        return [expr]
    
    out = []
    sub_idx = {sub : None for sub in _sub_cache}

    for factor in expr.args:
        
        subs_in_factor = list(sp.ordered({oper.sub for oper in factor.atoms(Operator)}))
                                    # NOTE: must use a set.
        
        if all([sub_idx[sub] is None for sub in subs_in_factor]):
            """
            The given argument can go into its own slot in 'out'
            since there is no sub shared with other items
            already in the output (those with specified value in 'sub_idx').
            """
            
            for sub in subs_in_factor:
                sub_idx[sub] = len(out)
            out.append(factor)
        
        else:
            """
            If there is a specified sub in 'factor', then the factor
            shares at least one sub with an already existing item
            in 'out'. As such, we find the first shared sub and
            multiply 'factor' into that slot. While we are at it,
            we can collect 'sub's whose value in 'sub_idx' is still
            'None' to assign their values to be the same as the
            specified 'sub', telling the algorithm that future
            factors with these 'sub's already have a shared slot in
            'out'. 
            
            Additionally, since there may be more than one
            specified 'sub' in 'factor', we can use a flag such that
            further enconters with specified 'sub's are ignored by
            the algorithm.
            """
            sub_already_with_slot_in_out = None
            subs_with_unspecified_idx = []
            for sub in subs_in_factor:
                if sub_idx[sub] is None:
                    subs_with_unspecified_idx.append(sub)
                elif sub_already_with_slot_in_out is None:
                    sub_already_with_slot_in_out = sub
                    out[sub_idx[sub_already_with_slot_in_out]] *= factor        
                    
            for ssub in subs_with_unspecified_idx:
                sub_idx[ssub] = sub_idx[sub_already_with_slot_in_out]
                
    return out

_collect_non_polynomial_by_sub(1)

[1]